In [106]:
import os
import sys
print(sys.path)
import BioSimSpace as BSS

['/opt/miniforge/envs/openbiosim/lib/python39.zip', '/opt/miniforge/envs/openbiosim/lib/python3.9', '/opt/miniforge/envs/openbiosim/lib/python3.9/lib-dynload', '', '/home/purohit.52@pharmacy.ohio-state.edu/.local/lib/python3.9/site-packages', '/opt/miniforge/envs/openbiosim/lib/python3.9/site-packages']


In [107]:
atp1 = BSS.IO.readMolecules(["inputs/ligands/adp-6t8b_C.mol2"])[0] 
atp1 = BSS.Parameters.gaff(atp1).getMolecule()
BSS.IO.saveMolecules("atp1", atp1, ["gro87", "grotop"])
atp2 = BSS.IO.readMolecules(["inputs/ligands/adp_maegher.prm7","inputs/ligands/adp_maegher.rst7"])[0]   
BSS.IO.saveMolecules("atp2", atp2, ["gro87", "grotop"])

['/home/purohit.52@pharmacy.ohio-state.edu/Desktop/RBFE/bss_fep_advanced_tut_bg-main/ATP-ADP_RBFE/atp2.gro',
 '/home/purohit.52@pharmacy.ohio-state.edu/Desktop/RBFE/bss_fep_advanced_tut_bg-main/ATP-ADP_RBFE/atp2.top']

In [108]:
mapping = BSS.Align.matchAtoms(atp1, atp2)
print(mapping)

{0: 1, 1: 3, 2: 0, 3: 2, 4: 5, 5: 6, 6: 7, 7: 4, 8: 8, 9: 9, 10: 12, 11: 14, 12: 31, 13: 33, 14: 35, 15: 37, 16: 15, 17: 17, 18: 18, 19: 20, 20: 21, 21: 22, 22: 23, 23: 26, 24: 27, 25: 29, 26: 30, 27: 11, 28: 10, 29: 13, 30: 32, 31: 34, 32: 36, 33: 38, 34: 16, 35: 19, 36: 25, 37: 24, 38: 28}


code to delete last three columns of atp2.gro and replace with last three columns of atp1.gro such that key (row) of atp1 matches with value (row) of atp2

In [109]:
!sed -i '$ d' atp2.gro

In [110]:
!sed -i 1,2d atp2.gro

In [111]:
!sed -i '$ d' atp1.gro

In [112]:
!sed -i 1,2d atp1.gro

In [113]:
import pandas as pd
atp2_df=pd.read_csv("atp2.gro",delimiter=r"\s+", header=None)
atp1_df=pd.read_csv("atp1.gro",delimiter=r"\s+", header=None)

In [114]:
for key in mapping:
    #print(key, "->", mapping[key])
    atp2_value=mapping[key]
    atp2_df.loc[atp2_value, 3]=atp1_df.loc[key, 3]
    atp2_df.loc[atp2_value, 4]=atp1_df.loc[key, 4]
    atp2_df.loc[atp2_value, 5]=atp1_df.loc[key, 5]
atp2_df.head()

0    1  2      3       4       5
0  1adp  O1B  1  9.107  12.259  11.857
1  1adp   PB  2  9.137  12.332  11.986
2  1adp  O2B  3  9.161  12.226  12.117
3  1adp  O3B  4  9.249  12.437  11.990
4  1adp  O3A  5  8.996  12.417  12.034

In [115]:
atp2_df.to_csv('output.txt', index=False, header=False, sep=' ')

In [116]:
!sed -i 's\1atp\1 atp\g' output.txt

In [117]:
!sed -i 's\1adp\1 adp\g' output.txt

In [118]:
!sed -i 's\1gtp\1 gtp\g' output.txt

In [119]:
!sed -i 's\1gdp\1 gdp\g' output.txt

In [120]:
def convert_to_gro(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Write the title for the .gro file
        outfile.write("generated\n")
        
        # Read all lines to count number of atoms
        lines = infile.readlines()
        number_of_atoms = len(lines)
        
        # Write the number of atoms in the .gro file
        outfile.write(f"{number_of_atoms}\n")
        
        for idx, line in enumerate(lines):
            columns = line.strip().split()
             
            # Adjust the columns as needed for your specific data
            res_num = int(columns[0])         # Residue number
            res_name = columns[1]             # Residue name
            atom_name = columns[2]            # Atom name
            atom_num = int(columns[3])        # Atom number
            x = float(columns[4])             # X coordinate
            y = float(columns[5])             # Y coordinate
            z = float(columns[6])             # Z coordinate
                        
            # Write the atom line in .gro format
            outfile.write(f"{res_num:5d}{res_name:<5}{atom_name:>5}{atom_num:5d}{x:8.3f}{y:8.3f}{z:8.3f}\n")
        
        # Write the box dimensions (example values, adjust as needed)
        outfile.write("   1.00000   1.00000   1.00000\n")

# Example usage
input_file = 'output.txt'
output_file = 'atp2.gro'
convert_to_gro(input_file, output_file)

In [121]:
!rm output.txt atp1.gro atp1.top

In [122]:
atp = BSS.IO.readMolecules(["atp2.top","atp2.gro"])[0]


In [123]:
protein = BSS.IO.readMolecules(["inputs/6t8b_withDNA/topol.top","inputs/6t8b_withDNA/step3_input.gro"])

In [124]:
complx = atp+protein 

In [125]:
complex_sol = BSS.Solvent.tip3p(molecule=complx, box=3*[160*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)


In [126]:
protocol = BSS.Protocol.Minimisation(steps=10)
process = BSS.MD.run(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="minim")
process.run()  
process.wait()
print("done")

done
